In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import os
import sys
sys.path.append('../')

from skeletor.data import loadImages

from pepe.preprocess import checkImageType

from skimage.morphology import skeletonize

from tqdm import tqdm

import dask.array as da
from dask_image.imread import imread
from dask_image.ndmeasure import label
from dask_image.ndfilters import convolve

import functools
import operator
import dask.dataframe as dd
import pickle

import open3d as o3d

import sparse

In [ ]:
dataFolder = '/home/jack/Workspaces/data/scans/2024-11-01_LG_C_PNG/'

images = loadImages(dataFolder, format='sparse')

images

In [ ]:
## Parameters
greenChannel = 1
dataFolder = '/home/jack/Workspaces/data/scans/2024-11-01_LG_C_PNG/'
imageExtension = "png"

start = 0
end = None
skip = 2
threshold = 10
dsFactor = 2
# End parameters

imagePaths = np.sort([f for f in os.listdir(dataFolder) if f[-3:].lower() == imageExtension.lower()])
imagePaths = np.array([os.path.join(dataFolder, f) for f in imagePaths if 'mask' not in f])[start:end:skip]

print(f'Found {len(imagePaths)} images')
testImg = checkImageType(imagePaths[0])

print(np.max(testImg))
imageData = np.zeros((len(imagePaths), *np.array((testImg.shape[:2]), dtype=np.int64)//dsFactor), dtype=np.uint8)

for i in tqdm(range(len(imagePaths))):
    binImage = np.mean(checkImageType(imagePaths[i + start]), axis=-1)
    binImage[binImage < threshold] = 0
    binImage[binImage > 0] = 1
    dsImage = binImage[::dsFactor,::dsFactor]
    imageData[i] = dsImage
    

print(imageData.shape)
print(np.max(imageData))

In [ ]:
with open('test_3d_image.npy', 'wb') as f:
    np.save(f, imageData)

In [ ]:
dataFolder = '/home/jack/Workspaces/data/scans/2024-11-01_LG_C_PNG/'
imageExtension = "png"
threshold = 50
dsFactor = 15

images = imread(f'{dataFolder}*.{imageExtension}')

# Grayscale
images = np.mean(images, axis=-1)

# Remove the mask
images = images[:-1]

images = images[::dsFactor,::dsFactor,::dsFactor]
images.shape

In [ ]:
import dask_image.ndfilters

smoothedImages = dask_image.ndfilters.gaussian_filter(images, sigma=2)

binImages = smoothedImages > threshold

binImages

In [ ]:
factor = np.array([4,10,12])  # even numbers
chunksize = np.array(binImages.shape)//factor

rechunkBinImages = binImages.rechunk(chunksize)

rechunkBinImages

In [ ]:
skel = da.map_overlap(skeletonize, rechunkBinImages)
ndim = 3

In [ ]:
skel.compute()

In [ ]:
# Label each non-zero pixel with a unique integer id
structure_kernel = np.zeros((3,) * ndim)
structure_kernel[(1,) * ndim] = 1  # add centre pixel
skelint, num_features = label(skel, structure=structure_kernel)

In [ ]:
# Label each non-zero pixel with the number of neighbors it has
degree_kernel = np.ones((3,) * ndim)
degree_kernel[(1,) * ndim] = 0  # remove centre pixel
degrees_image = convolve(skel.astype(int), degree_kernel, mode='constant') * skel

In [ ]:
# Mofified from slices_from_chunks from dask.array.core
from itertools import product
from dask.array.slicing import cached_cumsum


def slices_from_chunks_overlap(chunks, array_shape, depth=1):
    """Translate chunks tuple to a set of slices in product order

    Parameters
    ----------
    chunks : tuple
        The chunks of the corresponding dask array.
    array_shape : tuple
        Shape of the corresponding dask array.
    depth : int
        The number of pixels to overlap, providing we're not at the array edge.

    Example
    -------
    >>> slices_from_chunks_overlap(((4,), (7, 7)), (4, 14), depth=1)  # doctest: +NORMALIZE_WHITESPACE
     [(slice(0, 5, None), slice(0, 8, None)),
      (slice(0, 5, None), slice(6, 15, None))]
    """
    cumdims = [cached_cumsum(bds, initial_zero=True) for bds in chunks]

    slices = []
    for starts, shapes in zip(cumdims, chunks):
        inner_slices = []
        for s, dim, maxshape in zip(starts, shapes, array_shape):
            slice_start = s
            slice_stop = s + dim
            if slice_start > 0:
                slice_start -= depth
            if slice_stop >= maxshape:
                slice_stop += depth
            inner_slices.append(slice(slice_start, slice_stop))
        slices.append(inner_slices)
    
    return list(product(*slices))

In [ ]:
from dask.delayed import delayed
import pandas as pd
import numpy as np
import scipy

from skan.nputil import raveled_steps_to_neighbors
from skan.csr import _write_pixel_graph


@delayed
def skeleton_graph_func(skelint, spacing=1):
    ndim = skelint.ndim
    spacing = np.ones(ndim, dtype=float) * spacing
    num_edges = _num_edges(skelint.astype(bool))
    padded_skelint = np.pad(skelint, 1)  # pad image to prevent looparound errors
    steps, distances = raveled_steps_to_neighbors(padded_skelint.shape, ndim,
                                                  spacing=spacing)

    # from function skan.csr._pixel_graph
    row = np.empty(num_edges, dtype=int)
    col = np.empty(num_edges, dtype=int)
    data = np.empty(num_edges, dtype=float)
    k = _write_pixel_graph(padded_skelint, steps, distances, row, col, data)

    return pd.DataFrame({"row": row, "col": col, "data": data})
Alongside indexing assignment and retrieval, DOK arrays support any arbitrary broadcasting function to any number of arguments where the arguments can be SparseArray objects, scipy.sparse.spmatrix objects, or numpy.ndarrays.

x = sparse.random((10, 10), 0.5, format="dok")
y = sparse.random((10, 10), 0.5, format="dok")
sparse.elemwise(np.add, x, y)


def _num_edges(skel):
    degree_kernel = np.ones((3,) * ndim)
    degree_kernel[(1,) * ndim] = 0  # remove centre pixel
    degree_image = scipy.ndimage.convolve(skel.astype(int),
                                          degree_kernel,
                                          mode='constant') * skel
    num_edges = np.sum(degree_image)
    return int(num_edges)

In [ ]:
# Calculate the results we need to make the skeleton graph
image = skelint
block_iter = zip(
    np.ndindex(*image.numblocks),
    map(functools.partial(operator.getitem, image),
        slices_from_chunks_overlap(image.chunks, image.shape, depth=1))
)

meta = dd.utils.make_meta([('row', np.int64), ('col', np.int64), ('data', np.float64)])  # it's very important to include meta
intermediate_results = [dd.from_delayed(skeleton_graph_func(block), meta=meta) for _, block in block_iter]  # this appears to be triggering a lot of computation
results = dd.concat(intermediate_results)
results = results.drop_duplicates()

In [ ]:
# Create the skeleton graph adjacency matrix
k = len(results)
print(k)
row = np.array(results['row'])
col = np.array(results['col'])
data = np.array(results['data'])

graph = sparse.coo_matrix((data[:k], (row[:k], col[:k]))).tocsr()
graph

In [ ]:
print(float(skel[1][0][0]))

In [ ]:
with open('../2024-10-11_LG_A_PNG_sparse_skeleton.pckl', 'rb') as f:
    sparseImage = pickle.load(f)
    

In [ ]:
sparseImage.shape

In [ ]:
imagePoints = sparseImage.coords.T

pointCloudClean = o3d.geometry.PointCloud()
pointCloudClean.points = o3d.utility.Vector3dVector(imagePoints)
pointCloudClean.paint_uniform_color((1, 0, 0))

o3d.visualization.draw_geometries([pointCloudClean])